## Gradio
This is a tool that allows us to create nice user interfaces for our LLMs. It simplifies the task of deploying them by removing the need to understand web development in depth.

In [1]:
import os
import gradio as gr
import google.generativeai as palm
# Custom modules
from embeddings_palm import get_palm_embeddings

In [2]:
api_key = os.getenv("PALM_API_KEY")
palm.configure(api_key=api_key)

## Pinecone

In [3]:
import pinecone

### Credentials

In [8]:
pinecone_api_key = os.getenv('PINECONE_API_KEY_03')

'6ccca79b-2c14-4276-ab90-7d84b68f9250'

### Connecting to Pinecone Index

In [9]:

index_name = 'econwiki'

# initialize connection to pinecone
pinecone.init(
    api_key= pinecone_api_key,
    environment="gcp-starter"  # next to API key in console
)

In [10]:
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0192,
 'namespaces': {'': {'vector_count': 1920}},
 'total_vector_count': 1920}

In [11]:
def make_query(query):
    # Get query vector embeddings
    xq = get_palm_embeddings(query)
    res = index.query(xq, top_k=5, include_metadata=True)

    # get list of retrieved text
    contexts = [item['metadata']['text'] for item in res['matches']]

    # Concatenate retrieved texts from vector database with the query
    ## May exceed context length if too many
    augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query 
    
    # system message to 'prime' the model
    primer = f"""
        You are an econ major seeking to understand the world around you through economics \
        You will be provided with texts on economic phenomena on which you will make comments \
        These texts come from various sources such as articles, textbooks and so on \
        Your role is to condense the contents therein into language that is accessible to an intermediate student \
        You will not oversimplify key concepts but you will also not use inacessible technical language \
        After the text there will be a question you will be required to answer \
        If you do not find the answer in the text provided you will clearly state that you do not know \
        That is, you will not make answers up without validation from text \
        Provide direct quotations from key figures in the text to support your answers \
    """
    res = palm.chat(context=primer, messages=augmented_query, temperature=0.0)
    return res.last

In [12]:
demo = gr.Interface(fn=make_query, 
                    inputs=[gr.Textbox(label="Query to make", lines=6)],
                    outputs=[gr.Markdown(label="Result")],
                    title="EconWiki",
                    description="A central repository for macro economic data. Ask Away!"
                   )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://93e21c44ade01b7243.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
gr.close_all()

Closing server running on port: 7860
